# Frequently Asked Questions

Here we are attempting to answer some commonly asked questions that appear on Github, and Stack Overflow.

In [ ]:
import pandas as pd
import woodwork as ww

import featuretools as ft

## EntitySet

### How do I get a list of column names and types in an `EntitySet`?

After you create your `EntitySet`, you may wish to view the column names. An `EntitySet` contains multiple DataFrames, one for each table in the `EntitySet`.

In [ ]:
es = ft.demo.load_mock_customer(return_entityset=True)
es

If you want to view the underlying Dataframe, you can do the following:

In [ ]:
es["transactions"].head()

If you want view the columns and types for the "transactions" DataFrame, you can do the following:

In [ ]:
es["transactions"].ww

### What is the difference between `copy_columns` and `additional_columns`?
The function `normalize_dataframe` creates a new DataFrame and a relationship from unique values of an existing DataFrame. It takes 2 similar arguments:

- `additional_columns` removes columns from the base DataFrame and moves them to the new DataFrame. 
- `copy_columns` keeps the given columns in the base DataFrame, but also copies them to the new DataFrame.

In [ ]:
data = ft.demo.load_mock_customer()
transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])
products_df = data["products"]

es = ft.EntitySet(id="customer_data")
es = es.add_dataframe(
    dataframe_name="transactions",
    dataframe=transactions_df,
    index="transaction_id",
    time_index="transaction_time",
)

es = es.add_dataframe(
    dataframe_name="products", dataframe=products_df, index="product_id"
)

es = es.add_relationship("products", "product_id", "transactions", "product_id")

Before we normalize to create a new DataFrame, let's look at the base DataFrame

In [ ]:
es["transactions"].head()

Notice the columns `session_id`, `session_start`, `join_date`, `device`, `customer_id`, and `zip_code`.

In [ ]:
es = es.normalize_dataframe(
    base_dataframe_name="transactions",
    new_dataframe_name="sessions",
    index="session_id",
    make_time_index="session_start",
    additional_columns=["join_date"],
    copy_columns=["device", "customer_id", "zip_code", "session_start"],
)

Above, we normalized the columns to create a new DataFrame. 

- For `additional_columns`, the following column `['join_date]` will be removed from the `transactions` DataFrame, and moved to the new `sessions` DataFrame. 

- For `copy_columns`, the following columns `['device', 'customer_id', 'zip_code','session_start']` will be copied from the `transactions` DataFrame to the new `sessions` DataFrame. 

Let's see this in the actual `EntitySet`.

In [ ]:
es["transactions"].head()

Notice above how `['device', 'customer_id', 'zip_code','session_start']` are still in the `transactions` DataFrame, while `['join_date']` is not. But, they have all been moved to the `sessions` DataFrame, as seen below.

In [ ]:
es["sessions"].head()

### Why did my columns get new semantic tags?

During the creation of your `EntitySet`, you might be wondering why the semantic tags in your columns change.

In [ ]:
data = ft.demo.load_mock_customer()
transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])
products_df = data["products"]

es = ft.EntitySet(id="customer_data")
es = es.add_dataframe(
    dataframe_name="transactions",
    dataframe=transactions_df,
    index="transaction_id",
    time_index="transaction_time",
)
es.plot()

If a column contains semantic tags, they will appear on the right side of a semicolon in the plot above. Notice how `session_id` and `session_start` do not have any semantic tags currently associated to them.

Now, let's normalize the transactions DataFrame to create a new DataFrame.

In [ ]:
es = es.normalize_dataframe(
    base_dataframe_name="transactions",
    new_dataframe_name="sessions",
    index="session_id",
    make_time_index="session_start",
    additional_columns=["session_start"],
)
es.plot()

The `session_id` now has the sematic tag `foreign_key` in the `transactions` DataFrame, and `index` in the new DataFrame, `sessions`. This is the case because when we normalize the DataFrame, we create a new relationship between the `transactions` and `sessions`. There is a one to many relationship between the parent DataFrame, `sessions`, and child DataFrame, `transactions`.

Therefore, `session_id` has the semantic tag `foreign_key` in `transactions` because it represents an `index` in another DataFrame. There would be a similar effect if we added another DataFrame using `add_dataframe` and `add_relationship`. 

In addition, when we created the new DataFrame, we set `session_start` as the `time_index`. This added the semantic tag `time_index` to the `session_start` column in the new `sessions` DataFrame because it now represents a `time_index`.

### How do I update a column's description or metadata?

You can directly update the description or metadata attributes of the column schema. However, you must specifically use the column schema returned by `DataFrame.ww.columns['col_name']`, **not** `DataFrame.ww['col_name'].ww.schema`. The column schema from `DataFrame.ww.columns['col_name']` is still associated with the EntitySet and propagates any attribute updates, whereas the other does not. As an example, this is how you can update a column's description or metadata:

```python
column_schema = df.ww.columns['col_name']
column_schema.description = 'my description'
column_schema.metadata.update(key='value')
```


### How do I combine two or more interesting values?

You might want to create features that are conditioned on multiple values before they are calculated. This would require the use of `interesting_values`. However, since we are trying to create the feature with multiple conditions, we will need to modify the Dataframe before we create the `EntitySet`.

Let's look at how you might accomplish this. 

First, let's create our Dataframes.

In [ ]:
data = ft.demo.load_mock_customer()
transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])
products_df = data["products"]

In [ ]:
transactions_df.head()

In [ ]:
products_df.head()

Now, let's modify our `transactions` Dataframe to create the additional column that represents multiple conditions for our feature.

In [ ]:
transactions_df["product_id_device"] = (
    transactions_df["product_id"].astype(str) + " and " + transactions_df["device"]
)

Here, we created a new column called `product_id_device`, which just combines the `product_id` column, and the `device` column.

Now let's create our `EntitySet`.

In [ ]:
es = ft.EntitySet(id="customer_data")
es = es.add_dataframe(
    dataframe_name="transactions",
    dataframe=transactions_df,
    index="transaction_id",
    time_index="transaction_time",
    logical_types={
        "product_id": ww.logical_types.Categorical,
        "product_id_device": ww.logical_types.Categorical,
        "zip_code": ww.logical_types.PostalCode,
    },
)

es = es.add_dataframe(
    dataframe_name="products", dataframe=products_df, index="product_id"
)

es = es.normalize_dataframe(
    base_dataframe_name="transactions",
    new_dataframe_name="sessions",
    index="session_id",
    additional_columns=["device", "product_id_device", "customer_id"],
)

es = es.normalize_dataframe(
    base_dataframe_name="sessions", new_dataframe_name="customers", index="customer_id"
)
es

Now, we are ready to add our interesting values. 

First, let's view our options for what the interesting values could be.

In [ ]:
interesting_values = transactions_df["product_id_device"].unique().tolist()
interesting_values

If you wanted to, you could pick a subset of these, and the `where` features created would only use those conditions. In our example, we will use all the possible interesting values.

Here, we set all of these values as our interesting values for this specific DataFrame and column. If we wanted to, we could make interesting values in the same way for more than one column, but we will just stick with this one for this example.

In [ ]:
values = {"product_id_device": interesting_values}
es.add_interesting_values(dataframe_name="sessions", values=values)

Now we can run DFS.

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=["count"],
    where_primitives=["count"],
    trans_primitives=[],
)
feature_matrix.head()

To better understand the `where` clause features, let's examine one of those features. 
The feature `COUNT(sessions WHERE product_id_device = 5 and tablet)`, tells us how many sessions the customer purchased `product_id` 5 while on a tablet. Notice how the feature depends on multiple conditions **(product_id = 5 & device = tablet)**.

In [ ]:
feature_matrix[["COUNT(sessions WHERE product_id_device = 5 and tablet)"]]

## DFS

### Why is DFS not creating aggregation features?
You may have created your `EntitySet`, and then applied DFS to create features. However, you may be puzzled as to why no aggregation features were created. 

- **This is most likely because you have a single DataFrame in your EntitySet, and DFS is not capable of creating aggregation features with fewer than 2 DataFrames. Featuretools looks for a relationship, and aggregates based on that relationship.**

Let's look at a simple example.

In [ ]:
data = ft.demo.load_mock_customer()
transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])

es = ft.EntitySet(id="customer_data")
es = es.add_dataframe(
    dataframe_name="transactions", dataframe=transactions_df, index="transaction_id"
)
es

Notice how we only have 1 DataFrame in our `EntitySet`. If we try to create aggregation features on this `EntitySet`, it will not be possible because DFS needs 2 DataFrames to generate aggregation features. 

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es, target_dataframe_name="transactions"
)
feature_defs

None of the above features are aggregation features. To fix this issue, you can add another DataFrame to your `EntitySet`.

**Solution #1 - You can add new DataFrame if you have additional data.**

In [ ]:
products_df = data["products"]
es = es.add_dataframe(
    dataframe_name="products", dataframe=products_df, index="product_id"
)
es

Notice how we now have an additional DataFrame in our `EntitySet`, called `products`.

**Solution #2 - You can normalize an existing DataFrame.**

In [ ]:
es = es.normalize_dataframe(
    base_dataframe_name="transactions",
    new_dataframe_name="sessions",
    index="session_id",
    make_time_index="session_start",
    additional_columns=["device", "customer_id", "zip_code", "join_date"],
    copy_columns=["session_start"],
)
es

Notice how we now have an additional DataFrame in our `EntitySet`, called `sessions`. Here, the normalization created a relationship between `transactions` and `sessions`. However, we could have specified a relationship between `transactions` and `products` if we had only used Solution \#1.

Now, we can generate aggregation features.

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es, target_dataframe_name="transactions"
)
feature_defs[:-10]

A few of the aggregation features are:

- `<Feature: sessions.MAX(transactions.amount)>`
- `<Feature: sessions.SKEW(transactions.amount)>`
- `<Feature: sessions.MIN(transactions.amount)>`
- `<Feature: sessions.MEAN(transactions.amount)>`
- `<Feature: sessions.COUNT(transactions)>`

### How do I speed up the runtime of DFS?

One issue you may encounter while running `ft.dfs` is slow performance. While Featuretools has generally optimal default settings for calculating features, you may want to speed up performance when you are calculating on a large number of features. 

One quick way to speed up performance is by adjusting the `n_jobs` settings of `ft.dfs` or `ft.calculate_feature_matrix`.

```python
# setting n_jobs to -1 will use all cores

feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_dataframe_name="customers",
                                      n_jobs=-1)

                                      
feature_matrix, feature_defs = ft.calculate_feature_matrix(entityset=es,
                                                           features=feature_defs,
                                                           n_jobs=-1)
```


**For more ways to speed up performance, please visit:**

- [Improving Computational Performance](../guides/performance.ipynb#improving-computational-performance)

### How do I include only certain features when running DFS?

When using DFS to generate features, you may wish to include only certain features. There are multiple ways that you do this:

- Use `ignore_columns` to specify columns in a DataFrame that should not be used to create features. It is a dictionary mapping dataframe names to a list of column names to ignore.

- Use `drop_contains` to drop features that contain any of the strings listed in this parameter.

- Use `drop_exact` to drop features that exactly match any of the strings listed in this parameter.

Here is an example of using all three parameters:

In [ ]:
es = ft.demo.load_mock_customer(return_entityset=True)

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    ignore_columns={
        "transactions": ["amount"],
        "customers": ["age", "gender", "birthday"],
    },  # ignore these columns
    drop_contains=["customers.SUM("],  # drop features that contain these strings
    drop_exact=["STD(transactions.quanity)"],
)  # drop features that exactly match

### How do I specify primitives on a per column or per DataFrame basis?

When using DFS to generate features, you may wish to use only certain features or DataFrames for specific primitives. This can be done through the `primitive_options` parameter. The `primitive_options` parameter is a dictionary that maps a primitive or a tuple of primitives to a dictionary containing options for the primitive(s). A primitive or tuple of primitives can also be mapped to a list of option dictionaries if the primitive(s) 
takes multiple inputs. The primitive keys can be the string names of the primitive, the primitive class, or specific instances of the primitive. Each dictionary supplies options for their respective input column. There are multiple ways to control how primitives get applied through these options:

- Use `ignore_dataframes` to specify DataFrames that should not be used to create features for that primitive. It is a list of DataFrame names to ignore.

- Use `include_dataframes` to specify the only DataFrames to be included to create features for that primitive. It is a list of DataFrame names to include.

- Use `ignore_columns` to specify columns in a DataFrame that should not be used to create features for that primitive. It is a dictionary mapping a DataFrame name to a list of column names to ignore.

- Use `include_columns` to specify the only columns in a DataFrame that should be used to create features for that primitive. It is a dictionary mapping a DataFrame name to a list of column names to include.

You can also use `primitive_options` to specify which DataFrames or columns you wish to use as groupbys for groupby transformation primitives:

- Use `ignore_groupby_dataframes` to specify DataFrames that should not be used to get groupbys for that primitive. It is a list of DataFrame names to ignore.

- Use `include_groupby_dataframes` to specify the only DataFrames that should be used to get groupbys for that primitive. It is a list of DataFrame names to include.

- Use `ignore_groupby_columns` to specify columns in a DataFrame that should not be used as groupbys for that primitive. It is a dictionary mapping a DataFrame name to a list of column names to ignore.

- Use `include_groupby_columns` to specify the only columns in a DataFrame that should be used as groupbys for that primitive. It is a dictionary mapping a DataFrame name to a list of column names to include.

Here is an example of using some of these options:

In [ ]:
es = ft.demo.load_mock_customer(return_entityset=True)

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    primitive_options={
        "mode": {
            "ignore_dataframes": ["sessions"],
            "ignore_columns": {"products": ["brand"], "transactions": ["product_id"]},
        },
        # For mode, ignore the "sessions" DataFrame and only include "brands" in the
        # "products" dataframe and "product_id" in the "transactions" DataFrame
        ("count", "mean"): {"include_dataframes": ["sessions", "transactions"]},
        # For count and mean, only include the dataframes "sessions" and "transactions"
    },
)

Note that if options are given for a specific instance of a primitive and for the primitive generally (either by string name or class), the instances with their own options will not use the generic options. For example, in this case:
```
special_mean = Mean()
options = {
    special_mean: {'include_dataframes': ['customers']},
    'mean': {'include_dataframes': ['sessions']}
```
the primitive `special_mean` will not use the DataFrame `sessions` because it's options have it only include `customers`. Every other instance of the `Mean` primitive will use the `'mean'` options.  

**For more examples of specifying options for DFS, please visit:**

- [Specifying Primitive Options](../guides/specifying_primitive_options.rst)

### If I didn't specify the **cutoff_time**, what date will be used for the feature calculations?

The cutoff time will be set to the current time using `cutoff_time = datetime.now()`.

### How do I select a certain amount of past data when calculating features?

You may encounter a situation when you wish to make prediction using only a certain amount of historical data. You can accomplish this using the `training_window` parameter in `ft.dfs`. When you use the `training_window`, Featuretools will use the historical data between the `cutoff_time` and `cutoff_time - training_window`.

In order to make the calculation, Featuretools will check the time in the `time_index` column of the `target_dataframe`.

In [ ]:
es = ft.demo.load_mock_customer(return_entityset=True)
es["customers"].ww.time_index

Our target_dataframe has a `time_index`, which is needed for the `training_window` calculation. Here, we are creating a cutoff time DataFrame so that we can have a unique training window for each customer.

In [ ]:
cutoff_times = pd.DataFrame()
cutoff_times["customer_id"] = [1, 2, 3, 1]
cutoff_times["time"] = pd.to_datetime(
    ["2014-1-1 04:00", "2014-1-1 05:00", "2014-1-1 06:00", "2014-1-1 08:00"]
)
cutoff_times["label"] = [True, True, False, True]

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    cutoff_time=cutoff_times,
    cutoff_time_in_index=True,
    training_window="1 hour",
)
feature_matrix.head()

Above, we ran DFS with `training_window` argument of `1 hour` to create features that only used customer data collected in the last hour (from the cutoff time we provided).

### Can I run DFS on a single table? 

Although possible, running DFS on a single table doesn't make full use of DFS's capabilities. For one, DFS will not be able to use any aggregation primitives, which require at least two tables. You will only be able to use transform primitives. This limits the complexity of the features that DFS can generate through feature stacking. Additionally, in certain situations, running single table DFS on data with time columns could risk label leakage. With data split in multiple tables, featuretools can filter data based on the cutoff time instead of assuming data was flattened appropriately, but it can not do this with only a single table. 

If you only have a single table of data, DFS can certainly still be of use. There are two main ways to pass in a single table to DFS. 

The first is to simply create an EntitySet with one table. 

For example:

In [ ]:
transactions_df = ft.demo.load_mock_customer(return_single_table=True)

es = ft.EntitySet(id="customer_data")
es = es.add_dataframe(
    dataframe_name="transactions",
    dataframe=transactions_df,
    index="transaction_id",
    time_index="transaction_time",
)

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="transactions",
    trans_primitives=[
        "time_since",
        "day",
        "is_weekend",
        "cum_min",
        "minute",
        "weekday",
        "percentile",
        "year",
        "week",
        "cum_mean",
    ],
)

The second way is to insert the dataframe into a dictionary mapping its name to a tuple containing specific dataframe information. We then pass in that dictionary to the `dataframes` argument in DFS.

In this scenario, for the value in our dictionary, we pass in a tuple containing the dataframe, its index column, and its time index. More information about the possible parameters can be found in the [DFS documentation](https://featuretools.alteryx.com/en/stable/generated/featuretools.dfs.html#featuretools.dfs).

For example: 

In [ ]:
transactions_df = ft.demo.load_mock_customer(return_single_table=True)

dataframes = {"transactions": (transactions_df, "transaction_id", "transaction_time")}

feature_matrix, feature_defs = ft.dfs(
    dataframes=dataframes,
    target_dataframe_name="transactions",
    trans_primitives=[
        "time_since",
        "day",
        "is_weekend",
        "cum_min",
        "minute",
        "weekday",
        "percentile",
        "year",
        "week",
        "cum_mean",
    ],
)

Before we examine the output, let's look at our original single table.

In [ ]:
transactions_df.head()

Now we can look at the transformations that Featuretools was able to apply to this single DataFrame to create feature matrix.

In [ ]:
feature_matrix.head()

### How do I prevent label leakage with DFS?

One concern you might have with using DFS is about label leakage. You want to make sure that labels in your data aren't used incorrectly to create features and the feature matrix.

**Featuretools is particularly focused on helping users avoid label leakage.**

There are two ways to prevent label leakage depending on if your data has timestamps or not.

#### 1. Data without timestamps
In the case where you do not have timestamps, you can create one `EntitySet` using only the training data and then run `ft.dfs`. This will create a feature matrix using only the training data, but also return a list of feature definitions. Next, you can create an `EntitySet` using the test data and recalculate the same features by calling `ft.calculate_feature_matrix` with the list of feature definitions from before. 

Here is what that flow would look like:

First, let's create our training data.

In [ ]:
train_data = pd.DataFrame(
    {
        "customer_id": [1, 2, 3, 4, 5],
        "age": [40, 50, 10, 20, 30],
        "gender": ["m", "f", "m", "f", "f"],
        "signup_date": pd.date_range("2014-01-01 01:41:50", periods=5, freq="25min"),
        "labels": [True, False, True, False, True],
    }
)
train_data.head()

Now, we can create an entityset for our training data.

In [ ]:
es_train_data = ft.EntitySet(id="customer_train_data")
es_train_data = es_train_data.add_dataframe(
    dataframe_name="customers", dataframe=train_data, index="customer_id"
)
es_train_data

Next, we are ready to create our features, and feature matrix for the training data.  We don't want Featuretools to use the labels column to build new features, so we will use the ``ignore_columns`` option to exclude it.  This would also remove the labels column from the feature matrix, so we will tell DFS to include it as a seed feature.

In [ ]:
labels_feature = ft.Feature(es_train_data["customers"].ww["labels"])
feature_matrix_train, feature_defs = ft.dfs(
    entityset=es_train_data,
    target_dataframe_name="customers",
    ignore_columns={"customers": ["labels"]},
    seed_features=[labels_feature],
)
feature_matrix_train

We will also encode our feature matrix to make machine learning compatible features. 

In [ ]:
feature_matrix_train_enc, features_enc = ft.encode_features(
    feature_matrix_train, feature_defs
)
feature_matrix_train_enc.head()

Notice how the whole feature matrix only includes numeric and boolean values now.

Now we can use the feature definitions to calculate our feature matrix for the test data, and avoid label leakage.

In [ ]:
test_train = pd.DataFrame(
    {
        "customer_id": [6, 7, 8, 9, 10],
        "age": [20, 25, 55, 22, 35],
        "gender": ["f", "m", "m", "m", "m"],
        "signup_date": pd.date_range("2014-01-01 01:41:50", periods=5, freq="25min"),
        "labels": [True, False, False, True, True],
    }
)

es_test_data = ft.EntitySet(id="customer_test_data")
es_test_data = es_test_data.add_dataframe(
    dataframe_name="customers",
    dataframe=test_train,
    index="customer_id",
    time_index="signup_date",
)

# Use the feature definitions from earlier
feature_matrix_enc_test = ft.calculate_feature_matrix(
    features=features_enc, entityset=es_test_data
)

feature_matrix_enc_test.head()

Check out the [Modeling](frequently_asked_questions.ipynb#Modeling) section for an example of using the encoded matrix with sklearn.

#### 2. Data with timestamps

If your data has timestamps, the best way to prevent label leakage is to use a list of **cutoff times**, which specify the last point in time data is allowed to be used for each row in the resulting feature matrix. To use **cutoff times**, you need to set a time index for each time sensitive DataFrame in your entity set.

> **Tip: Even if your data doesn’t have time stamps, you could add a column with dummy timestamps that can be used by Featuretools as time index.**

When you call `ft.dfs`, you can provide a DataFrame of cutoff times like this:

In [ ]:
cutoff_times = pd.DataFrame(
    {
        "customer_id": [1, 2, 3, 4, 5],
        "time": pd.date_range("2014-01-01 01:41:50", periods=5, freq="25min"),
    }
)
cutoff_times.head()

In [ ]:
train_test_data = pd.DataFrame(
    {
        "customer_id": [1, 2, 3, 4, 5],
        "age": [20, 25, 55, 22, 35],
        "gender": ["f", "m", "m", "m", "m"],
        "signup_date": pd.date_range("2010-01-01 01:41:50", periods=5, freq="25min"),
    }
)

es_train_test_data = ft.EntitySet(id="customer_train_test_data")
es_train_test_data = es_train_test_data.add_dataframe(
    dataframe_name="customers",
    dataframe=train_test_data,
    index="customer_id",
    time_index="signup_date",
)

feature_matrix_train_test, features = ft.dfs(
    entityset=es_train_test_data,
    target_dataframe_name="customers",
    cutoff_time=cutoff_times,
    cutoff_time_in_index=True,
)
feature_matrix_train_test.head()

Above, we have created a feature matrix that uses cutoff times to avoid label leakage. We could also encode this feature matrix using `ft.encode_features`.

### What is the difference between passing a primitive object versus a string to DFS?  

There are 2 ways to pass primitives to DFS: the primitive object, or a string of the primitive name. 

We will use the Transform primitive called `TimeSincePrevious` to illustrate the differences.

First, let's use the string of primitive name.

In [ ]:
es = ft.demo.load_mock_customer(return_entityset=True)

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=[],
    trans_primitives=["time_since_previous"],
)
feature_matrix

Now, let's use the primitive object.

In [ ]:
from featuretools.primitives import TimeSincePrevious

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=[],
    trans_primitives=[TimeSincePrevious],
)
feature_matrix

As we can see above, the feature matrix is the same.

However, if we need to modify controllable parameters in the primitive, we should use the primitive object. 
For instance, let's make TimeSincePrevious return units of hours (the default is in seconds).

In [ ]:
from featuretools.primitives import TimeSincePrevious

time_since_previous_in_hours = TimeSincePrevious(unit="hours")

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=[],
    trans_primitives=[time_since_previous_in_hours],
)
feature_matrix

## Features

### How can I select features based on some attributes (a specific string, an explicit primitive type, a return type, a given depth)?

You may wish to select a subset of your features based on some attributes. 

Let's say you wanted to select features that had the string `amount` in its name. You can check for this by using the `get_name` function on the feature definitions.

In [ ]:
es = ft.demo.load_mock_customer(return_entityset=True)

feature_defs = ft.dfs(
    entityset=es, target_dataframe_name="customers", features_only=True
)

features_with_amount = []
for x in feature_defs:
    if "amount" in x.get_name():
        features_with_amount.append(x)
features_with_amount[0:5]

You might also want to only select features that are aggregation features.

In [ ]:
from featuretools import AggregationFeature

features_only_aggregations = []
for x in feature_defs:
    if type(x) == AggregationFeature:
        features_only_aggregations.append(x)
features_only_aggregations[0:5]

Also, you might only want to select features that are calculated at a certain depth. You can do this by using the `get_depth` function. 

In [ ]:
features_only_depth_2 = []
for x in feature_defs:
    if x.get_depth() == 2:
        features_only_depth_2.append(x)
features_only_depth_2[0:5]

Finally, you might only want features that return a certain type. You can do this by using the `column_schema` attribute. For more information on working with column schemas, take a look at [Transitioning from Variables to Woodwork](transition_to_ft_v1.0.ipynb).

In [ ]:
features_only_numeric = []
for x in feature_defs:
    if "numeric" in x.column_schema.semantic_tags:
        features_only_numeric.append(x)
features_only_numeric[0:5]

Once you have your specific feature list, you can use `ft.calculate_feature_matrix` to generate a feature matrix for only those features.

For our example, let's use the features with only the string `amount` in its name.

In [ ]:
feature_matrix = ft.calculate_feature_matrix(
    entityset=es, features=features_with_amount
)  # change to your specific feature list
feature_matrix.head()

Above, notice how all the column names for our feature matrix contain the string `amount`.

### How do I create **where** features?

Sometimes, you might want to create features that are conditioned on a second value before it is calculated. This extra filter is called a “where clause”. You can create these features using the using the `interesting_values` of a column.

If you have categorical columns in your `EntitySet`, you can use `add_interesting_values`. This function will  find interesting values for your categorical columns, which can then be used to generate “where” clauses.

First, let's create our `EntitySet`.

In [ ]:
es = ft.demo.load_mock_customer(return_entityset=True)
es

Now we can add the interesting values for the categorical column.

In [ ]:
es.add_interesting_values()

Now we can run DFS with the `where_primitives` argument to define which primitives to apply with where clauses. In this case, let's use the primitive `count`. For this to work, the primitive `count` must be present in both `agg_primitives` and `where_primitives`.

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=["count"],
    where_primitives=["count"],
    trans_primitives=[],
)
feature_matrix.head()

We have now created some useful features. One example of a useful feature is the `COUNT(sessions WHERE device = tablet)`. This feature tells us how many sessions a customer completed on a tablet.

In [ ]:
feature_matrix[["COUNT(sessions WHERE device = tablet)"]]

## Primitives

### What is the difference between the primitive types (Transform, GroupBy Transform, & Aggregation)?

You might curious to know the difference between the primitive groups.
Let's review the differences between transform, groupby transform, and aggregation primitives.

First, let's create a simple `EntitySet`.

In [ ]:
import pandas as pd

import featuretools as ft

df = pd.DataFrame(
    {
        "id": [1, 2, 3, 4, 5, 6],
        "time_index": pd.date_range("1/1/2019", periods=6, freq="D"),
        "group": ["a", "a", "a", "a", "a", "a"],
        "val": [5, 1, 10, 20, 6, 23],
    }
)
es = ft.EntitySet()
es = es.add_dataframe(
    dataframe_name="observations", dataframe=df, index="id", time_index="time_index"
)

es = es.normalize_dataframe(
    base_dataframe_name="observations", new_dataframe_name="groups", index="group"
)

es.plot()

After calling `normalize_dataframe`, the column "group" has the semantic tag "foreign_key" because it identifies another DataFrame. Alternatively, it could be set using the `semantic_tags` parameter when we first call `es.add_dataframe()`.

#### Transform Primitive

The cum_sum primitive calculates the running sum in list of numbers.

In [ ]:
from featuretools.primitives import CumSum

cum_sum = CumSum()
cum_sum([1, 2, 3, 4, 5]).tolist()

If we apply it using the `trans_primitives` argument it will calculate it over the entire observations DataFrame like this:

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    target_dataframe_name="observations",
    entityset=es,
    agg_primitives=[],
    trans_primitives=["cum_sum"],
    groupby_trans_primitives=[],
)

feature_matrix

#### Groupby Transform Primitive

If we apply it using `groupby_trans_primitives`, then DFS will first group by any foreign key columns before applying the transform primitive. As a result, we get the cumulative sum by group.

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    target_dataframe_name="observations",
    entityset=es,
    agg_primitives=[],
    trans_primitives=[],
    groupby_trans_primitives=["cum_sum"],
)

feature_matrix

#### Aggregation Primitive

Finally, there is also the aggregation primitive "sum". If we use sum, it will calculate the sum for the group at the cutoff time for each row. Because we didn't specify a cutoff time it will use all the data for each group for each row.

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    target_dataframe_name="observations",
    entityset=es,
    agg_primitives=["sum"],
    trans_primitives=[],
    cutoff_time_in_index=True,
    groupby_trans_primitives=[],
)

feature_matrix

If we set the cutoff time of each row to be the time index, then use sum as an aggregation primitive, the result is the same as cum_sum. (Though the order is different in the displayed dataframe).

In [ ]:
cutoff_time = df[["id", "time_index"]]
cutoff_time

In [ ]:
feature_matrix, feature_defs = ft.dfs(
    target_dataframe_name="observations",
    entityset=es,
    agg_primitives=["sum"],
    trans_primitives=[],
    groupby_trans_primitives=[],
    cutoff_time_in_index=True,
    cutoff_time=cutoff_time,
)

feature_matrix

### How do I get a list of all Aggregation and Transform primitives?

You can do `featuretools.list_primitives()` to get all the primitive in Featuretools. It will return a DataFrame with the names, type, and description of the primitives.

In [ ]:
df_primitives = ft.list_primitives()
df_primitives.head()

In [ ]:
df_primitives.tail()

### How do I change the units for a TimeSince primitive?
There are a few primitives in Featuretools that make some time-based calculation. These include `TimeSince, TimeSincePrevious, TimeSinceLast, TimeSinceFirst`. 

You can change the units from the default seconds to any valid time unit, by doing the following:

In [ ]:
from featuretools.primitives import (
    TimeSince,
    TimeSinceFirst,
    TimeSinceLast,
    TimeSincePrevious,
)

time_since = TimeSince(unit="minutes")
time_since_previous = TimeSincePrevious(unit="hours")
time_since_last = TimeSinceLast(unit="days")
time_since_first = TimeSinceFirst(unit="years")

es = ft.demo.load_mock_customer(return_entityset=True)

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    agg_primitives=[time_since_last, time_since_first],
    trans_primitives=[time_since, time_since_previous],
)

Above, we changed the units to the following:
- minutes for `TimeSince`
- hours for `TimeSincePrevious`
- days for `TimeSinceLast`
- years for `TimeSinceFirst`.


Now we can see that our feature matrix contains multiple features where the units for the TimeSince primitives are changed.

In [ ]:
feature_matrix.head()

There are now features where time unit is different from the default of seconds, such as `TIME_SINCE_LAST(sessions.session_start, unit=days)`, and `TIME_SINCE_FIRST(sessions.session_start, unit=years)`.

## Modeling

### How does my train & test data work with Featuretools and sklearn's **train_test_split**?

You might be wondering how to properly use your train & test data with Featuretools, and sklearn's **train_test_split**. There are a few things you must do to ensure accuracy with this workflow.

Let's imagine we have a Dataframes for our train data, with the labels.

In [ ]:
train_data = pd.DataFrame(
    {
        "customer_id": [1, 2, 3, 4, 5],
        "age": [20, 25, 55, 22, 35],
        "gender": ["f", "m", "m", "m", "m"],
        "signup_date": pd.date_range("2010-01-01 01:41:50", periods=5, freq="25min"),
        "labels": [False, True, True, False, False],
    }
)
train_data.head()

Now we can create our `EntitySet` for the train data, and create our features. To prevent label leakage, we will use cutoff times (see [earlier question](#How-do-I-prevent-label-leakage-with-DFS?)).

In [ ]:
es_train_data = ft.EntitySet(id="customer_data")
es_train_data = es_train_data.add_dataframe(
    dataframe_name="customers", dataframe=train_data, index="customer_id"
)

cutoff_times = pd.DataFrame(
    {
        "customer_id": [1, 2, 3, 4, 5],
        "time": pd.date_range("2014-01-01 01:41:50", periods=5, freq="25min"),
    }
)

feature_matrix_train, features = ft.dfs(
    entityset=es_train_data,
    target_dataframe_name="customers",
    cutoff_time=cutoff_times,
    cutoff_time_in_index=True,
)
feature_matrix_train.head()

We will also encode our feature matrix to compatible for machine learning algorithms.

In [ ]:
feature_matrix_train_enc, feature_enc = ft.encode_features(
    feature_matrix_train, features
)
feature_matrix_train_enc.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = feature_matrix_train_enc.drop(["labels"], axis=1)
y = feature_matrix_train_enc["labels"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

Now you can use the encoded feature matrix with sklearn's **train_test_split**. This will allow you to train your model, and tune your parameters.

### How are categorical columns encoded when splitting training and testing data?

You might be wondering what happens when categorical columns are encoded with your training and testing data. You might be curious to know what happens if the train data has a categorical column that is not present in the testing data. 

Let's explore a simple example to see what happens during the encoding process.

In [ ]:
train_data = pd.DataFrame(
    {
        "customer_id": [1, 2, 3, 4, 5],
        "product_purchased": ["coke zero", "car", "toothpaste", "coke zero", "car"],
    }
)
es_train = ft.EntitySet(id="customer_data")
es_train = es_train.add_dataframe(
    dataframe_name="customers",
    dataframe=train_data,
    index="customer_id",
    logical_types={"product_purchased": ww.logical_types.Categorical},
)
feature_matrix_train, features = ft.dfs(
    entityset=es_train, target_dataframe_name="customers"
)
feature_matrix_train

We will use `ft.encode_features` to properly encode the `product_purchased` column.

In [ ]:
feature_matrix_train_encoded, features_encoded = ft.encode_features(
    feature_matrix_train, features
)
feature_matrix_train_encoded.head()

Now lets imagine we have some test data that has doesn't have one of the categorical values (**toothpaste**). Also, the test data has a value that wasn't present in the train data (**water**).

In [ ]:
test_data = pd.DataFrame(
    {
        "customer_id": [6, 7, 8, 9, 10],
        "product_purchased": ["coke zero", "car", "coke zero", "coke zero", "water"],
    }
)

es_test = ft.EntitySet(id="customer_data")
es_test = es_test.add_dataframe(
    dataframe_name="customers", dataframe=test_data, index="customer_id"
)

feature_matrix_test = ft.calculate_feature_matrix(
    entityset=es_test, features=features_encoded
)
feature_matrix_test.head()

As seen above, we were able to successfully handle the encoding, and deal with the following complications: 
- **toothpaste** was present in the training data but not present in the testing data 
- **water** was present in the test data but not present in the training data. 

## Errors & Warnings

### Why am I getting this error 'Index is not unique on dataframe'?
You may be trying to create your `EntitySet`, and run into this error. 
```python
IndexError: Index column must be unique
```
**This is because each dataframe in your EntitySet needs a unique index.**

Let's look at a simple example.

In [ ]:
product_df = pd.DataFrame({"id": [1, 2, 3, 4, 4], "rating": [3.5, 4.0, 4.5, 1.5, 5.0]})
product_df

Notice how the `id` column has a duplicate index of `4`. If you try to add this dataframe to the EntitySet, you will run into the following error.

```python
es = ft.EntitySet(id="product_data")
es = es.add_dataframe(dataframe_name="products",
                      dataframe=product_df,
                      index="id")
```

```
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
<ipython-input-78-854fbaf207f8> in <module>
      1 es = ft.EntitySet(id="product_data")
----> 2 es = es.add_dataframe(dataframe_name="products",
      3                       dataframe=product_df,
      4                       index="id")

~/Code/featuretools/featuretools/entityset/entityset.py in add_dataframe(self, dataframe, dataframe_name, index, logical_types, semantic_tags, make_index, time_index, secondary_time_index, already_sorted)
    625             index_was_created, index, dataframe = _get_or_create_index(index, make_index, dataframe)
    626 
--> 627             dataframe.ww.init(name=dataframe_name,
    628                               index=index,
    629                               time_index=time_index,

/usr/local/Caskroom/miniconda/base/envs/featuretools/lib/python3.8/site-packages/woodwork/table_accessor.py in init(self, index, time_index, logical_types, already_sorted, schema, validate, use_standard_tags, **kwargs)
     94         """
     95         if validate:
---> 96             _validate_accessor_params(self._dataframe, index, time_index, logical_types, schema, use_standard_tags)
     97         if schema is not None:
     98             self._schema = schema

/usr/local/Caskroom/miniconda/base/envs/featuretools/lib/python3.8/site-packages/woodwork/table_accessor.py in _validate_accessor_params(dataframe, index, time_index, logical_types, schema, use_standard_tags)
    877         # We ignore these parameters if a schema is passed
    878         if index is not None:
--> 879             _check_index(dataframe, index)
    880         if logical_types:
    881             _check_logical_types(dataframe.columns, logical_types)

/usr/local/Caskroom/miniconda/base/envs/featuretools/lib/python3.8/site-packages/woodwork/table_accessor.py in _check_index(dataframe, index)
    903         # User specifies an index that is in the dataframe but not unique
--> 904         raise IndexError('Index column must be unique')
    905 
    906 

IndexError: Index column must be unique
```

To fix the above error, you can do one of the following solutions:

**Solution #1 - You can create a unique index on your Dataframe.**

In [ ]:
product_df = pd.DataFrame({"id": [1, 2, 3, 4, 5], "rating": [3.5, 4.0, 4.5, 1.5, 5.0]})
product_df

Notice how we now have a unique index column called `id`.

In [ ]:
es = es.add_dataframe(dataframe_name="products", dataframe=product_df, index="id")
es

As seen above, we can now create our DataFrame for our `EntitySet` without an error by creating a unique index in our Dataframe.

**Solution #2 - Set make_index to True in your call to add_dataframe to create a new index on that data**
- `make_index` creates a unique index for each row by just looking at what number the row is, in relation to all the other rows.

In [ ]:
product_df = pd.DataFrame({"id": [1, 2, 3, 4, 4], "rating": [3.5, 4.0, 4.5, 1.5, 5.0]})

es = ft.EntitySet(id="product_data")
es = es.add_dataframe(
    dataframe_name="products", dataframe=product_df, index="product_id", make_index=True
)

es["products"]

As seen above, we created our dataframe for our `EntitySet` without an error using the `make_index` argument.

### Why am I getting the following warning 'Using training_window but last_time_index is not set'?

If you are using a training window, and you haven't set a `last_time_index` for your dataframe, you will get this warning.
The training window attribute in Featuretools limits the amount of past data that can be used while calculating a particular feature vector.

You can add the `last_time_index` to all dataframes automatically by calling `your_entityset.add_last_time_indexes()` after you create your `EntitySet`. This will remove the warning.

In [ ]:
es = ft.demo.load_mock_customer(return_entityset=True)
es.add_last_time_indexes()

Now we can run DFS without getting the warning.

In [ ]:
cutoff_times = pd.DataFrame()
cutoff_times["customer_id"] = [1, 2, 3, 1]
cutoff_times["time"] = pd.to_datetime(
    ["2014-1-1 04:00", "2014-1-1 05:00", "2014-1-1 06:00", "2014-1-1 08:00"]
)
cutoff_times["label"] = [True, True, False, True]

feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    cutoff_time=cutoff_times,
    cutoff_time_in_index=True,
    training_window="1 hour",
)

#### last_time_index vs. time_index

- The `time_index` is when the instance was first known.
- The `last_time_index` is when the instance appears for the last time.
- For example, a customer’s session has multiple transactions which can happen at different points in time. If we are trying to count the number of sessions a user has in a given time period, we often want to count all the sessions that had any transaction during the training window. To accomplish this, we need to not only know when a session starts (**time_index**), but also when it ends (**last_time_index**). The last time that an instance appears in the data is stored as the `last_time_index` of a dataframe. 
- Once the last_time_index has been set, Featuretools will check to see if the last_time_index is after the start of the training window. That, combined with the cutoff time, allows DFS to discover which data is relevant for a given training window.

### Why am I getting errors with Featuretools on [Google Colab](https://colab.research.google.com/)?

[Google Colab](https://colab.research.google.com/), by default, has Featuretools `0.4.1` installed. You may run into issues following our newest guides, or latest documentation while using an older version of Featuretools. Therefore, we suggest you upgrade to the latest featuretools version by doing the following in your notebook in Google Colab:
```shell
!pip install -U featuretools
```

You may need to Restart the runtime by doing **Runtime** -> **Restart Runtime**.
You can check latest Featuretools version by doing following:
```python
import featuretools as ft
print(ft.__version__)
```
You should see a version greater than `0.4.1`